# Guided Project: Building a Database for Crime Reports

### Introduction
In this guided project, we will build a database for storing Boston crime data. The data set we will be using for this project is a boston.csv file, provided by Dataquest. We will be creating two user groups; readonly and readwrite. 
- readonly: Users in this group will have permission to read data only.
- readwrite: Users in this group will have permissions to read and alter data but cannot delete tables.

### Creating the database and the schema

In [1]:
import psycopg2
conn = psycopg2.connect(dbname="dq", user="dq")
# Set autocommit to True
# Needed to create databases
conn.autocommit = True
cur = conn.cursor()
# Create the crime_db database
cur.execute("CREATE DATABASE crime_db;")
conn.close()

In [2]:
# Connect to the crime_db datebase
conn = psycopg2.connect(dbname="crime_db", user="dq")
conn.autocommit = True
cur = conn.cursor()
# Create the crimes schema
cur.execute("CREATE SCHEMA crimes;")

### Obtaining column names and samples
- Obtain the header row and assign it to a variable named col_headers
- Obtain the first data row and assign it to a variable named first_row

In [3]:
import csv
with open("boston.csv") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

### Creating a function for analyzing column values
We will create a function that given the name of a CSV file and a column index, computes the set of all distinct values in that column.

In [4]:
def get_col_set(csv_filename, col_index):
    import csv
    values = set()
    with open(csv_filename, "r") as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values))

incident_number 298329
offense_code 219
description 239
date 1177
day_of_the_week 7
lat 18177
long 18177


### Analyzing the maximum length of the description column

In [ ]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [ ]:
descriptions = get_col_set("boston.csv", 2)
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

### Creating the table

In [ ]:
print(col_headers)
print(first_row)

In [ ]:
cur.execute('''
    CREATE TYPE weekday AS ENUM (
        'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
        );
''')
cur.execute('''
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat DECIMAL,
        long DECIMAL
    );
''')

### Load the data into the table
Now that we have created the table, we can use cur.copy_expert() to load the data.

In [ ]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
cur.execute("SELECT * FROM crimes.boston_crimes")
print(len(cur.fetchall()))

### Create the two user groups: readonly and readwrite

### Revoke public privileges
By following the least privilege principle, the first stop is to make sure that there are no privileges inherited from the public group and on the public schema. 

In [ ]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

### Creating the read only group
We will first create the readonly group, which gives users the privilege to only perform SELECT queries.

In [ ]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

### Creating the read write group
Now we will create the readwrite group, which gives users the privilege to perform SELECT, INSERT, DELETE and UPDATE queries. 

In [ ]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

### Creating one user for each group

In [ ]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

### Test the database setup

In [ ]:
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
cur.execute('''
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
''')
for user in cur:
    print(user)
print('\n')

cur.execute('''
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
''')
for user in cur:
    print(user)
conn.close()